In [94]:
import numpy as np
from scipy import linalg
import matplotlib.pyplot as plt
import time
from sklearn.datasets import fetch_openml
import pandas as pd
from sklearn.preprocessing import StandardScaler
import os

from metric import residual_error
from utility import seed_everything, frobenius_norm_sq
from load_data import load_dataset
from svd import svd_error
from random_css import random_css
from greedy_css import greedy_css, greedy_recursive_css, partition_greedy_css

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [33]:
seed_everything()

## Load data from dataset

In [71]:
dataset_name = "sonar"
dataset_dir = "datasets"

config_path = os.path.join(dataset_dir, dataset_name, "detail.yaml")
data_matrix = load_dataset(dataset_name, config_path, read_label=False)

Reading data from datasets/sonar/sonar.csv...
Data read done with shape (208, 60)


## Baseline -- best rank-k approximation using SVD

In [77]:
k = 5

baseline = svd_error(data_matrix, k)
baseline

np.float64(100.77146899491107)

In [74]:
def rank_k_svd(A, k):
    d, n = A.shape
    true_max_rank = min(d, n)
    k = min(k, true_max_rank)
    u, sigma, vh = np.linalg.svd(A)
    u = u[:, :k]
    sigma = np.diag(sigma[:k])
    vh = vh[:k]
    return u @ sigma @ vh

baseline_1 = frobenius_norm_sq(data_matrix - rank_k_svd(data_matrix, k))
baseline_1

np.float64(100.77146899491117)

In [76]:
U_svd, s_svd, Vt_svd = np.linalg.svd(data_matrix, full_matrices=False)
# 最优k维子空间投影误差 (由SVD的前k个左奇异向量张成的空间)
A_projected_to_optimal_k_space = U_svd[:, :k] @ (U_svd[:, :k].T @ data_matrix)
optimal_k_subspace_residual_sq = frobenius_norm_sq(data_matrix - A_projected_to_optimal_k_space)
optimal_k_subspace_residual_sq

np.float64(100.77146899491117)

## Random

In [78]:
indices_random = random_css(data_matrix, k)
error_random = residual_error(data_matrix, indices_random)
print(f"Selected Indices: {indices_random}, Residual Error: {error_random}")

Selected Indices: [6, 40, 48, 8, 9], Residual Error: 475.7822000962927


## Greedy

In [100]:
indices_greedy = greedy_css(data_matrix, k)
indices_greedy_recursive = greedy_recursive_css(data_matrix, k)
indices_greedy_partition = partition_greedy_css(data_matrix, k, k)

In [101]:
error_greedy = residual_error(data_matrix, indices_greedy)
error_greedy_recursive = residual_error(data_matrix, indices_greedy_recursive)
error_greedy_partition = residual_error(data_matrix, indices_greedy_partition)

In [102]:
print(error_greedy, error_greedy_recursive, error_greedy_partition)

151.83737430295722 151.83737430295722 156.98481066846742


## LSCSS

In [89]:
from css import lscss_algorithm

t = 2 * k
error_lscss = lscss_algorithm(data_matrix, k, t)

In [88]:
from demo import lscss_algorithm_1

_, selected_indices = lscss_algorithm_1(data_matrix, k, t)
error_lscss_1 = residual_error(data_matrix, selected_indices)

--- LSCSS 算法开始 ---
步骤 1-11: 初始化和构造 A'
  t=1: 在原始 A 上选择 k 列用于计算 alpha...
  计算得到 alpha = 9.136e-03
  已构造扰动矩阵 A' (形状: (208, 60))
  t=2: 在 A' 上选择 k 列作为局部搜索的初始解...
  局部搜索的初始列索引 (在 A' 中): [17, 22, 32, 38, 40]

步骤 13-15: 执行局部搜索 10 次...
--- LSCSS 算法结束 ---


In [90]:
print(error_lscss, error_lscss_1)

([18, 23, 30, 34, 40], np.float64(163.93528490259783)) 159.76754313634657
